# Visualization
## Streamflow Benchmarking Tutorial

This notebook is an example of how a HyTEST user may examine some streamflow benchmark results from a hydrologic model. 

Here, we are viewing daily streamflow benchmark results from the [example analysis](02_Analysis_StdSuite.ipynb) 
of [National Water Model Retrospective version 2.1](https://registry.opendata.aws/nwm-archive/), 
forced with AORC, at streamflow benchmark locations ("cobalt gages" [Foks et al., 2022](https://www.sciencebase.gov/catalog/item/6181ac65d34e9f2789e44897)).

Two different benchmark results are examined:
* the standard statistical suite results ('standard suite')
([Towler et al., 2022](https://www.sciencebase.gov/catalog/item/62336af9d34ec9f19eeb48fd)) 
* decomposition statistical suite results ('d-score') ([Hodson et al., 2022](https://www.sciencebase.gov/catalog/item/61d4c9e9d34ed79293fe91b4)).

## Step 0: Load Required Python Libraries

In [ ]:
try:
    import pandas as pd
    import holoviews as hv
    import hvplot.pandas
    import panel as pn
    from geoviews import tile_sources as gvts
    from holoviews import opts
except ImportError:
    print("A required library could not be found. ")
    raise

## Step 1: Load Domain Data

Loading the dataset representing the 'cobalt' gages. This includes location and other metadata about each streamflow gage. We will need this extra metadata to get lat/lon and other characteristics of the gage locations.

In [ ]:
cobalt = pd.read_csv(
    'https://www.sciencebase.gov/catalog/file/get/6181ac65d34e9f2789e44897?f=__disk__22%2F1a%2Fd2%2F221ad2fe9d95de17731ad35d0fc6b417a4b53ee1',
    dtype={'site_no':str, 'huc_cd':str, 'reachcode':str, 'comid':str, 'gagesII_class':str, 'aggecoregion': str}, 
    index_col='site_no')
# Re-format the gage_id/site_no string value.  ex:   "1000000"  ==> "USGS-1000000"
cobalt.rename(index=lambda x: f'USGS-{x}', inplace=True)
cobalt.rename(columns={'dec_lat_va':'Lat', 'dec_long_va':'Lon'} , inplace=True)
print(f"{len(cobalt.index)} gages in this benchmark")
cobalt.head(2)

## Step 2: Load results from the analysis notebooks

This table is the collection of evaluation metrics comparing observed vs. modeled streamflow of the cobalt gages. This data is the result
of the [Standard Suite Analysis Notebook](02_Analysis_StdSuite.ipynb).

In [ ]:
results = pd.read_csv(r'./Standard_streamflow_example.csv', dtype={'site_no':str} ).set_index('site_no', drop=False)

In [ ]:
# Merge benchmarks with cobalt data to form a single table, indexed by site_no
metrics = results.columns.tolist()[1:] #list of columns, EXCEPT the first column (site_no)
results = results.merge(
    cobalt,                # Table to merge with NWM
    how='left',            # left join preserves only records which have an index in modeled data dataframe.
    left_index=True,        
    right_index=True
    )
# The effect of the left join is that if a cobalt gage does not have a computed benchmark in NWM, 
# (e.g. due to error in the analysis process), it is dropped from the visualization set.

## Step 3: Visualize NWM Benchmark Results
A quick look at the joined data table shows that we have a collection of metrics, and the associated metadata, 
all indexed by the `site_id`.  This table is sortable by column -- you can scroll and select to explore the 
raw result.

In [ ]:
results.hvplot.table(
    columns=['site_no', 'drain_sqkm','KGE', 'NSE','logNSE','pearson','spearman','rSD','pbias','pBiasFMS','pBiasFHV','pBiasFLV','complete_yrs','n_days'], 
    sortable=True, 
    selectable=True
)

### Step 3a: Benchmark Results Mapped Over the Spatial Extent of the Conterminous United States

In [ ]:
var_select = pn.widgets.Select(name='Metric', options=metrics, value='pearson')
base_map_select = pn.widgets.Select(name='Basemap:', 
                                    options=list(gvts.tile_sources.keys()), 
                                    value='OSM')

@pn.depends(var=var_select, base_map=base_map_select)
def plot(var, base_map):
    return results.hvplot.points(x='Lon', y='Lat', color=var, cmap='turbo_r', geo=True, tiles=base_map)

col = pn.Column(var_select, base_map_select, plot)
col.servable('Hydro Assessment Tool')

### Step 3b: Boxplots
Grouped by 
* Gages-II classification, 
* HUC02 group, or 
* Aggregated Ecoregion

In [ ]:
# define which columns are metrics in the widget and which ones are groups
var_select = pn.widgets.Select(name='Metric', 
        options=metrics, 
        value='pearson')

group_select = pn.widgets.Select(name='Group By:', 
        options=['huc02', 'gagesII_class', 'aggecoregion'], 
        value='aggecoregion')

@pn.depends(var_select, group_select)
def plot(var, group):
        # local scope func to calc summary stats using var/group
        # build tooltip using the result. 
        # pandas will compute with groupby(). 
    return results.hvplot.box(y = var, by = group, height=400, width=800, legend=False)

col = pn.Column(var_select, group_select, plot)
col.servable('Benchmark Box Plots')

### Step 3c: Histograms 
Grouped by 
* Gages-II classification, 
* HUC02 group, or 
* Aggregated Ecoregion

In [ ]:
var_select = pn.widgets.Select(name='Metric', 
        options=metrics, 
        value='pearson'
    )

group_select = pn.widgets.Select(name='Group By:', 
        options=['huc02', 'gagesII_class', 'aggecoregion'], 
        value='aggecoregion'
    )

@pn.depends(var_select, group_select)
def plot(var, group):
    return results.hvplot.hist(var, group, subplots=True, width=400, bins = 500, legend='top')

col = pn.Column(var_select, group_select, plot)
col.servable('Benchmark Histograms')
# TODO: Constrain to fixed width for rendering.

### Step 3d: Metric by Latitude & Longitude

In [ ]:
var_select = pn.widgets.Select(name='Metric', options=metrics, 
                               value='pearson')

group_select = pn.widgets.Select(name='Group By:', 
                                    options=['Lon', 'Lat'], 
                                    value='Lon')

@pn.depends(var_select, group_select)
def plot(var, group):
    return results.hvplot.scatter(x=group, y=var, height=400, width = 500, legend='top', hover_cols=["site_no","Lat","Lon"])

col = pn.Column(var_select, group_select, plot)
col.servable('Lat/Lon Scatter Plot')

### Step 3e: Metric v. Metric

In [ ]:
var_select = pn.widgets.Select(name='Metric', options=metrics, 
                               value='pearson')

var2_select = pn.widgets.Select(name='Metric:', 
                                    options=metrics, 
                                    value='spearman')

@pn.depends(var_select, var2_select)

def plot(var, var2):
    return results.hvplot.scatter(x = var, y = var2, height=400, width = 500, legend='top', hover_cols=['site_no','Lat', 'Lon'])

col = pn.Column(var_select, var2_select, plot)
col.servable('Metric v Metric Scatter Plot')

------
## Step 4: Visualize D-Score Analysis Results

The above steps are repeated for the DScore analysis.

In [ ]:
#Data is loaded 
DScore = pd.read_csv(r'./DScore_streamflow_example.csv', dtype={'site_no':str} ).set_index('site_no', drop=False)

# Merge benchmarks with cobalt data to form a single table, indexed by site_no
metrics = DScore.columns.tolist()[1:] 
DScore = DScore.merge(
    cobalt, 
    how='left',
    left_index=True, 
    right_index=True
    )

In [ ]:
# Examine the data table
cols = ['site_no']
cols.extend(metrics)
DScore.hvplot.table(
    columns=cols, 
    sortable=True, 
    selectable=True
)

In [ ]:
# Score components as a percent of the total error (mse)

# Select only numeric columns for division (also first 15 columns)
numeric_columns = DScore.iloc[:, :16].select_dtypes(include='number')

# Perform the division by 'mse'
# Here we ensure to divide each numeric column by the 'mse' column
percentage_card = (numeric_columns.div(DScore['mse'], axis=0) * 100).round().astype(int)

# Merge back the metadata for mapping/etc
percentage_card = percentage_card.merge(
    cobalt, 
    how='left',
    left_index=True, 
    right_index=True
    )

# Set the name for the DataFrame
percentage_card.name = "Percent"

# Display the resulting DataFrame
print("\nPercentage Card DataFrame:")
print(percentage_card[0:5])

In [ ]:
# Note that a scorecard such as this is typically applied to a composite of d-score metrics
# computed for many gages (i.e., average of each component over all streamgages in our analysis).

# Calculate the average scores per column (components)
average_scores = percentage_card.iloc[:, :15].mean()
average_scores

## Map dscore components
Using the percentage of the MSE that a particular component is comprised of.

In [ ]:
# Map
var_select = pn.widgets.Select(name='Metric', options=metrics, value='mse')
base_map_select = pn.widgets.Select(name='Basemap:', 
                                    options=list(gvts.tile_sources.keys()), 
                                    value='OSM')

@pn.depends(var=var_select, base_map=base_map_select)
def plot(var, base_map):
    return percentage_card.hvplot.points(x='Lon', y='Lat', color=var, cmap='turbo_r', geo=True, tiles=base_map)

col = pn.Column(var_select, base_map_select, plot)
col.servable('Hydro Assessment Tool')

In [ ]:
# Box Plots
var_select = pn.widgets.Select(name='Metric', 
        options=metrics, 
        value='mse')

group_select = pn.widgets.Select(name='Group By:', 
        options=['huc02', 'gagesII_class', 'aggecoregion'], 
        value='aggecoregion')

@pn.depends(var_select, group_select)
def plot(var, group):
    return percentage_card.hvplot.box(y = var, by = group, height=400, width=800, legend=False)

col = pn.Column(var_select, group_select, plot)
col.servable('DScore Benchmark Box Plots')

In [ ]:
# metric vs metric
var_select = pn.widgets.Select(name='Metric', options=metrics, 
                               value='mse')

var2_select = pn.widgets.Select(name='Metric:', 
                                    options=metrics, 
                                    value='high')

@pn.depends(var_select, var2_select)

def plot(var, var2):
    return percentage_card.hvplot.scatter(x = var, y = var2, height=400, width = 500, legend='top', hover_cols=['site_no','Lat', 'Lon'])

col = pn.Column(var_select, var2_select, plot)
col.servable('Metric v Metric Scatter Plot')